In [1]:
from google.colab import files

In [2]:
# Upload dataset CSV
uploaded = files.upload()

Saving synthesis_dataset_bmi_refined.csv to synthesis_dataset_bmi_refined.csv


In [3]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Dataset loading 📊

In [4]:
import pandas as pd
from datasets import Dataset

# Load dataset
df = pd.read_csv('synthesis_dataset_bmi_refined.csv')

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

In [5]:
dataset

Dataset({
    features: ['Input', 'Target'],
    num_rows: 40000
})

# Model 🛵

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer dan model
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
def preprocess_function(examples):
    inputs = tokenizer(examples['Input'], max_length=512, truncation=True, padding='max_length')
    targets = tokenizer(examples['Target'], max_length=512, truncation=True, padding='max_length')
    inputs['labels'] = targets['input_ids']
    return inputs

# Tokenize dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

In [8]:
import torch

from transformers import Trainer, TrainingArguments

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    # evaluation_strategy="epoch",
    evaluation_strategy="no",  # Disable evaluation
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=1000,
    save_total_limit=2,
)

# Set up trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

if torch.cuda.is_available():
    model.to('cuda')

# Start fine-tuning
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.934500
1000,0.015500
1500,0.009700
2000,0.008200
2500,0.006900
3000,0.005500
3500,0.004900
4000,0.004500
4500,0.004300
5000,0.004100


TrainOutput(global_step=30000, training_loss=0.019575682187080383, metrics={'train_runtime': 6048.1731, 'train_samples_per_second': 19.841, 'train_steps_per_second': 4.96, 'total_flos': 1.624101617664e+16, 'train_loss': 0.019575682187080383, 'epoch': 3.0})

# Inference 💻

## Controlled Arrhythmia🔵



### Underweight, Indoor 🏠

#### Female 1

In [ ]:
# {'Age': 43, 'Gender': 'Female', 'Weight_kg': 45.9, 'Height_m': 1.58, 'BMI': 18.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 89, 'Systolic_BP': 136, 'Diastolic_BP': 84, 'Oxygen_Saturation': 96.3, 'Activity_Preference': 'indoor'}
input_text = "{'Age': 43, 'Gender': 'Female', 'Weight_kg': 45.9, 'Height_m': 1.58, 'BMI': 18.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 89, 'Systolic_BP': 136, 'Diastolic_BP': 84, 'Oxygen_Saturation': 96.3, 'Activity_Preference': 'indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stationary cycling', 'Activity_Duration': '15-20 minutes'


In [ ]:
# {'Age': 43, 'Gender': 'Female', 'Weight_kg': 45.9, 'Height_m': 1.58, 'BMI': 18.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 89, 'Systolic_BP': 136, 'Diastolic_BP': 84, 'Oxygen_Saturation': 96.3, 'Activity_Preference': 'indoor'}
input_text = "{'Age': 43, 'Gender': 'Female', 'Weight_kg': 45.9, 'Height_m': 1.58, 'BMI': 18.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 89, 'Systolic_BP': 136, 'Diastolic_BP': 84, 'Oxygen_Saturation': 96.3, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '15-20 minutes'


In [ ]:
# {'Age': 43, 'Gender': 'Female', 'Weight_kg': 45.9, 'Height_m': 1.58, 'BMI': 18.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 89, 'Systolic_BP': 136, 'Diastolic_BP': 84, 'Oxygen_Saturation': 96.3, 'Activity_Preference': 'indoor'}
input_text = "{'Age': 43, 'Gender': 'Female', 'Weight_kg': 45.9, 'Height_m': 1.58, 'BMI': 18.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 89, 'Systolic_BP': 136, 'Diastolic_BP': 84, 'Oxygen_Saturation': 96.3, 'Activity_Preference': 'indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Resistance band exercises', 'Activity_Duration': '15-20 minutes'


#### Female 2

In [13]:
# {'Age': 51, 'Gender': 'Female', 'Weight_kg': 56.0, 'Height_m': 1.79, 'BMI': 17.5, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 68, 'Systolic_BP': 127, 'Diastolic_BP': 73, 'Oxygen_Saturation': 96.8, 'Activity_Preference': 'Indoor'}
input_text = "{'Age': 51, 'Gender': 'Female', 'Weight_kg': 56.0, 'Height_m': 1.79, 'BMI': 17.5, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 68, 'Systolic_BP': 127, 'Diastolic_BP': 73, 'Oxygen_Saturation': 96.8, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Walking on treadmill', 'Activity_Duration': '15-20 minutes'


In [14]:
# {'Age': 51, 'Gender': 'Female', 'Weight_kg': 56.0, 'Height_m': 1.79, 'BMI': 17.5, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 68, 'Systolic_BP': 127, 'Diastolic_BP': 73, 'Oxygen_Saturation': 96.8, 'Activity_Preference': 'Indoor'}
input_text = "{'Age': 51, 'Gender': 'Female', 'Weight_kg': 56.0, 'Height_m': 1.79, 'BMI': 17.5, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 68, 'Systolic_BP': 127, 'Diastolic_BP': 73, 'Oxygen_Saturation': 96.8, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Breathing exercises', 'Activity_Duration': '15-20 minutes'


In [19]:
# {'Age': 51, 'Gender': 'Female', 'Weight_kg': 56.0, 'Height_m': 1.79, 'BMI': 17.5, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 68, 'Systolic_BP': 127, 'Diastolic_BP': 73, 'Oxygen_Saturation': 96.8, 'Activity_Preference': 'Indoor'}
input_text = "{'Age': 51, 'Gender': 'Female', 'Weight_kg': 56.0, 'Height_m': 1.79, 'BMI': 17.5, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 68, 'Systolic_BP': 127, 'Diastolic_BP': 73, 'Oxygen_Saturation': 96.8, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stationary cycling', 'Activity_Duration': '15-20 minutes'


In [20]:
# {'Age': 51, 'Gender': 'Female', 'Weight_kg': 56.0, 'Height_m': 1.79, 'BMI': 17.5, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 68, 'Systolic_BP': 127, 'Diastolic_BP': 73, 'Oxygen_Saturation': 96.8, 'Activity_Preference': 'Indoor'}
input_text = "{'Age': 51, 'Gender': 'Female', 'Weight_kg': 56.0, 'Height_m': 1.79, 'BMI': 17.5, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 68, 'Systolic_BP': 127, 'Diastolic_BP': 73, 'Oxygen_Saturation': 96.8, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stationary cycling', 'Activity_Duration': '15-20 minutes'


#### Female 3

In [21]:
# {'Age': 73, 'Gender': 'Female', 'Weight_kg': 38.0, 'Height_m': 1.52, 'BMI': 16.5, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Mild Pulmonary Hypertension', 'Heart_Rate': 70, 'Systolic_BP': 133, 'Diastolic_BP': 73, 'Oxygen_Saturation': 95.6, 'Activity_Preference': 'Indoor'},
input_text = "{'Age': 73, 'Gender': 'Female', 'Weight_kg': 38.0, 'Height_m': 1.52, 'BMI': 16.5, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 70, 'Systolic_BP': 133, 'Diastolic_BP': 73, 'Oxygen_Saturation': 95.6, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Walking on treadmill', 'Activity_Duration': '15-20 minutes'


In [22]:
# {'Age': 73, 'Gender': 'Female', 'Weight_kg': 38.0, 'Height_m': 1.52, 'BMI': 16.5, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Mild Pulmonary Hypertension', 'Heart_Rate': 70, 'Systolic_BP': 133, 'Diastolic_BP': 73, 'Oxygen_Saturation': 95.6, 'Activity_Preference': 'Indoor'},
input_text = "{'Age': 73, 'Gender': 'Female', 'Weight_kg': 38.0, 'Height_m': 1.52, 'BMI': 16.5, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 70, 'Systolic_BP': 133, 'Diastolic_BP': 73, 'Oxygen_Saturation': 95.6, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '15-20 minutes'


In [23]:
# {'Age': 73, 'Gender': 'Female', 'Weight_kg': 38.0, 'Height_m': 1.52, 'BMI': 16.5, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Mild Pulmonary Hypertension', 'Heart_Rate': 70, 'Systolic_BP': 133, 'Diastolic_BP': 73, 'Oxygen_Saturation': 95.6, 'Activity_Preference': 'Indoor'},
input_text = "{'Age': 73, 'Gender': 'Female', 'Weight_kg': 38.0, 'Height_m': 1.52, 'BMI': 16.5, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 70, 'Systolic_BP': 133, 'Diastolic_BP': 73, 'Oxygen_Saturation': 95.6, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stationary cycling', 'Activity_Duration': '15-20 minutes'


#### Male 1

In [24]:
# {'Age': 57, 'Gender': 'Male', 'Weight_kg': 47.2, 'Height_m': 1.73, 'BMI': 15.8, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 85, 'Systolic_BP': 139, 'Diastolic_BP': 77, 'Oxygen_Saturation': 97.0, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 57, 'Gender': 'Male', 'Weight_kg': 47.2, 'Height_m': 1.73, 'BMI': 15.8, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 85, 'Systolic_BP': 139, 'Diastolic_BP': 77, 'Oxygen_Saturation': 97.0, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Breathing exercises', 'Activity_Duration': '15-20 minutes'


In [25]:
# {'Age': 57, 'Gender': 'Male', 'Weight_kg': 47.2, 'Height_m': 1.73, 'BMI': 15.8, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 85, 'Systolic_BP': 139, 'Diastolic_BP': 77, 'Oxygen_Saturation': 97.0, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 57, 'Gender': 'Male', 'Weight_kg': 47.2, 'Height_m': 1.73, 'BMI': 15.8, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 85, 'Systolic_BP': 139, 'Diastolic_BP': 77, 'Oxygen_Saturation': 97.0, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Breathing exercises', 'Activity_Duration': '15-20 minutes'


In [26]:
# {'Age': 57, 'Gender': 'Male', 'Weight_kg': 47.2, 'Height_m': 1.73, 'BMI': 15.8, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 85, 'Systolic_BP': 139, 'Diastolic_BP': 77, 'Oxygen_Saturation': 97.0, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 57, 'Gender': 'Male', 'Weight_kg': 47.2, 'Height_m': 1.73, 'BMI': 15.8, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 85, 'Systolic_BP': 139, 'Diastolic_BP': 77, 'Oxygen_Saturation': 97.0, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stationary cycling', 'Activity_Duration': '15-20 minutes'


#### Male 2

In [27]:
# {'Age': 48, 'Gender': 'Male', 'Weight_kg': 47.5, 'Height_m': 1.63, 'BMI': 17.8, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 68, 'Systolic_BP': 116, 'Diastolic_BP': 87, 'Oxygen_Saturation': 96.1, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 48, 'Gender': 'Male', 'Weight_kg': 47.5, 'Height_m': 1.63, 'BMI': 17.8, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 68, 'Systolic_BP': 116, 'Diastolic_BP': 87, 'Oxygen_Saturation': 96.1, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '15-20 minutes'


In [28]:
# {'Age': 48, 'Gender': 'Male', 'Weight_kg': 47.5, 'Height_m': 1.63, 'BMI': 17.8, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 68, 'Systolic_BP': 116, 'Diastolic_BP': 87, 'Oxygen_Saturation': 96.1, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 48, 'Gender': 'Male', 'Weight_kg': 47.5, 'Height_m': 1.63, 'BMI': 17.8, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 68, 'Systolic_BP': 116, 'Diastolic_BP': 87, 'Oxygen_Saturation': 96.1, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '15-20 minutes'


In [29]:
# {'Age': 48, 'Gender': 'Male', 'Weight_kg': 47.5, 'Height_m': 1.63, 'BMI': 17.8, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 68, 'Systolic_BP': 116, 'Diastolic_BP': 87, 'Oxygen_Saturation': 96.1, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 48, 'Gender': 'Male', 'Weight_kg': 47.5, 'Height_m': 1.63, 'BMI': 17.8, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 68, 'Systolic_BP': 116, 'Diastolic_BP': 87, 'Oxygen_Saturation': 96.1, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Walking on treadmill', 'Activity_Duration': '15-20 minutes'


#### Male 3

In [31]:
input_text = "{'Age': 35, 'Gender': 'Male', 'Weight_kg': 47.5, 'Height_m': 1.63, 'BMI': 17.8, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 68, 'Systolic_BP': 116, 'Diastolic_BP': 87, 'Oxygen_Saturation': 96.1, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Resistance band exercises', 'Activity_Duration': '15-20 minutes'


In [33]:
input_text = "{'Age': 35, 'Gender': 'Male', 'Weight_kg': 47.5, 'Height_m': 1.63, 'BMI': 17.8, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 68, 'Systolic_BP': 116, 'Diastolic_BP': 87, 'Oxygen_Saturation': 96.1, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '15-20 minutes'


In [34]:
input_text = "{'Age': 35, 'Gender': 'Male', 'Weight_kg': 47.5, 'Height_m': 1.63, 'BMI': 17.8, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 68, 'Systolic_BP': 116, 'Diastolic_BP': 87, 'Oxygen_Saturation': 96.1, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Breathing exercises', 'Activity_Duration': '15-20 minutes'


### Underweight, Outdoor 🏞

#### Female 1

In [35]:
# {'Age': 49, 'Gender': 'Female', 'Weight_kg': 36.3, 'Height_m': 1.69, 'BMI': 12.7, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 62, 'Systolic_BP': 131, 'Diastolic_BP': 74, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Easy hiking on flat trails', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 49, 'Gender': 'Female', 'Weight_kg': 36.3, 'Height_m': 1.69, 'BMI': 12.7, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 62, 'Systolic_BP': 131, 'Diastolic_BP': 74, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Walking in the park', 'Activity_Duration': '15-20 minutes'


In [10]:
# {'Age': 49, 'Gender': 'Female', 'Weight_kg': 36.3, 'Height_m': 1.69, 'BMI': 12.7, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 62, 'Systolic_BP': 131, 'Diastolic_BP': 74, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Easy hiking on flat trails', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 49, 'Gender': 'Female', 'Weight_kg': 36.3, 'Height_m': 1.69, 'BMI': 12.7, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 62, 'Systolic_BP': 131, 'Diastolic_BP': 74, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Easy hiking on flat trails', 'Activity_Duration': '15-20 minutes'


In [12]:
# {'Age': 49, 'Gender': 'Female', 'Weight_kg': 36.3, 'Height_m': 1.69, 'BMI': 12.7, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 62, 'Systolic_BP': 131, 'Diastolic_BP': 74, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Easy hiking on flat trails', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 49, 'Gender': 'Female', 'Weight_kg': 36.3, 'Height_m': 1.69, 'BMI': 12.7, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 62, 'Systolic_BP': 131, 'Diastolic_BP': 74, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Light cycling', 'Activity_Duration': '15-20 minutes'


#### Female 2

In [39]:
input_text = "{'Age': 31, 'Gender': 'Female', 'Weight_kg': 41.4, 'Height_m': 1.69, 'BMI': 14.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 84, 'Systolic_BP': 138, 'Diastolic_BP': 81, 'Oxygen_Saturation': 97.5, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Easy hiking on flat trails', 'Activity_Duration': '15-20 minutes'


In [40]:
input_text = "{'Age': 31, 'Gender': 'Female', 'Weight_kg': 41.4, 'Height_m': 1.69, 'BMI': 14.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 84, 'Systolic_BP': 138, 'Diastolic_BP': 81, 'Oxygen_Saturation': 97.5, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Walking in the park', 'Activity_Duration': '15-20 minutes'


In [43]:
input_text = "{'Age': 31, 'Gender': 'Female', 'Weight_kg': 41.4, 'Height_m': 1.69, 'BMI': 14.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 84, 'Systolic_BP': 138, 'Diastolic_BP': 81, 'Oxygen_Saturation': 97.5, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Tai Chi in the open air', 'Activity_Duration': '15-20 minutes'


#### Male 1

In [36]:
# {'Age': 31, 'Gender': 'Male', 'Weight_kg': 41.4, 'Height_m': 1.69, 'BMI': 14.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 84, 'Systolic_BP': 138, 'Diastolic_BP': 81, 'Oxygen_Saturation': 97.5, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Walking in the park', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 31, 'Gender': 'Male', 'Weight_kg': 41.4, 'Height_m': 1.69, 'BMI': 14.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 84, 'Systolic_BP': 138, 'Diastolic_BP': 81, 'Oxygen_Saturation': 97.5, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Light cycling', 'Activity_Duration': '15-20 minutes'


In [37]:
# {'Age': 31, 'Gender': 'Male', 'Weight_kg': 41.4, 'Height_m': 1.69, 'BMI': 14.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 84, 'Systolic_BP': 138, 'Diastolic_BP': 81, 'Oxygen_Saturation': 97.5, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Walking in the park', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 31, 'Gender': 'Male', 'Weight_kg': 41.4, 'Height_m': 1.69, 'BMI': 14.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 84, 'Systolic_BP': 138, 'Diastolic_BP': 81, 'Oxygen_Saturation': 97.5, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Walking in the park', 'Activity_Duration': '15-20 minutes'


In [38]:
# {'Age': 31, 'Gender': 'Male', 'Weight_kg': 41.4, 'Height_m': 1.69, 'BMI': 14.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 84, 'Systolic_BP': 138, 'Diastolic_BP': 81, 'Oxygen_Saturation': 97.5, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Walking in the park', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 31, 'Gender': 'Male', 'Weight_kg': 41.4, 'Height_m': 1.69, 'BMI': 14.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 84, 'Systolic_BP': 138, 'Diastolic_BP': 81, 'Oxygen_Saturation': 97.5, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '15-20 minutes'


#### Male 2


In [44]:
# {'Age': 35, 'Gender': 'Male', 'Weight_kg': 57.0, 'Height_m': 1.79, 'BMI': 17.7, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 94, 'Systolic_BP': 119, 'Diastolic_BP': 88, 'Oxygen_Saturation': 97.0, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Easy hiking on flat trails', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 35, 'Gender': 'Male', 'Weight_kg': 57.0, 'Height_m': 1.79, 'BMI': 17.7, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 94, 'Systolic_BP': 119, 'Diastolic_BP': 88, 'Oxygen_Saturation': 97.0, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Light cycling', 'Activity_Duration': '15-20 minutes'


In [45]:
# {'Age': 35, 'Gender': 'Male', 'Weight_kg': 57.0, 'Height_m': 1.79, 'BMI': 17.7, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 94, 'Systolic_BP': 119, 'Diastolic_BP': 88, 'Oxygen_Saturation': 97.0, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Easy hiking on flat trails', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 35, 'Gender': 'Male', 'Weight_kg': 57.0, 'Height_m': 1.79, 'BMI': 17.7, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 94, 'Systolic_BP': 119, 'Diastolic_BP': 88, 'Oxygen_Saturation': 97.0, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '15-20 minutes'


In [46]:
# {'Age': 35, 'Gender': 'Male', 'Weight_kg': 57.0, 'Height_m': 1.79, 'BMI': 17.7, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 94, 'Systolic_BP': 119, 'Diastolic_BP': 88, 'Oxygen_Saturation': 97.0, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Easy hiking on flat trails', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 35, 'Gender': 'Male', 'Weight_kg': 57.0, 'Height_m': 1.79, 'BMI': 17.7, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 94, 'Systolic_BP': 119, 'Diastolic_BP': 88, 'Oxygen_Saturation': 97.0, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Easy hiking on flat trails', 'Activity_Duration': '15-20 minutes'


### Normal. Indoor 🏠

#### Male 1

In [47]:
# {'Age': 62, 'Gender': 'Male', 'Weight_kg': 66.6, 'Height_m': 1.89, 'BMI': 18.7, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 123, 'Diastolic_BP': 75, 'Oxygen_Saturation': 96.6, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 62, 'Gender': 'Male', 'Weight_kg': 66.6, 'Height_m': 1.89, 'BMI': 18.7, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 123, 'Diastolic_BP': 75, 'Oxygen_Saturation': 96.6, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Resistance band exercises', 'Activity_Duration': '30-45 minutes'


In [48]:
# {'Age': 62, 'Gender': 'Male', 'Weight_kg': 66.6, 'Height_m': 1.89, 'BMI': 18.7, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 123, 'Diastolic_BP': 75, 'Oxygen_Saturation': 96.6, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 62, 'Gender': 'Male', 'Weight_kg': 66.6, 'Height_m': 1.89, 'BMI': 18.7, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 123, 'Diastolic_BP': 75, 'Oxygen_Saturation': 96.6, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stationary cycling', 'Activity_Duration': '30-45 minutes'


In [49]:
# {'Age': 62, 'Gender': 'Male', 'Weight_kg': 66.6, 'Height_m': 1.89, 'BMI': 18.7, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 123, 'Diastolic_BP': 75, 'Oxygen_Saturation': 96.6, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 62, 'Gender': 'Male', 'Weight_kg': 66.6, 'Height_m': 1.89, 'BMI': 18.7, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 123, 'Diastolic_BP': 75, 'Oxygen_Saturation': 96.6, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '30-45 minutes'


In [52]:
#Invalid Input
input_text = "age:25, gender:male, weight:66kg, height:1.98"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

:25


In [53]:
#Invalid Input
input_text = "age:25, gender:male, weight:66kg, height:1.98"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

#### Male 2

In [54]:
# {'Age': 49, 'Gender': 'Male', 'Weight_kg': 63.2, 'Height_m': 1.73, 'BMI': 21.2, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 70, 'Systolic_BP': 128, 'Diastolic_BP': 88, 'Oxygen_Saturation': 98.1, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Breathing exercises', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 49, 'Gender': 'Male', 'Weight_kg': 63.2, 'Height_m': 1.73, 'BMI': 21.2, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 70, 'Systolic_BP': 128, 'Diastolic_BP': 88, 'Oxygen_Saturation': 98.1, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Breathing exercises', 'Activity_Duration': '30-45 minutes'


In [55]:
# {'Age': 49, 'Gender': 'Male', 'Weight_kg': 63.2, 'Height_m': 1.73, 'BMI': 21.2, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 70, 'Systolic_BP': 128, 'Diastolic_BP': 88, 'Oxygen_Saturation': 98.1, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Breathing exercises', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 49, 'Gender': 'Male', 'Weight_kg': 63.2, 'Height_m': 1.73, 'BMI': 21.2, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 70, 'Systolic_BP': 128, 'Diastolic_BP': 88, 'Oxygen_Saturation': 98.1, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '30-45 minutes'


In [56]:
# {'Age': 49, 'Gender': 'Male', 'Weight_kg': 63.2, 'Height_m': 1.73, 'BMI': 21.2, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 70, 'Systolic_BP': 128, 'Diastolic_BP': 88, 'Oxygen_Saturation': 98.1, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Breathing exercises', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 49, 'Gender': 'Male', 'Weight_kg': 63.2, 'Height_m': 1.73, 'BMI': 21.2, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 70, 'Systolic_BP': 128, 'Diastolic_BP': 88, 'Oxygen_Saturation': 98.1, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Breathing exercises', 'Activity_Duration': '30-45 minutes'


#### Female 1

In [57]:
# {'Age': 51, 'Gender': 'Female', 'Weight_kg': 61.8, 'Height_m': 1.77, 'BMI': 19.8, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 92, 'Systolic_BP': 136, 'Diastolic_BP': 74, 'Oxygen_Saturation': 98.4, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 51, 'Gender': 'Female', 'Weight_kg': 61.8, 'Height_m': 1.77, 'BMI': 19.8, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 92, 'Systolic_BP': 136, 'Diastolic_BP': 74, 'Oxygen_Saturation': 98.4, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stationary cycling', 'Activity_Duration': '30-45 minutes'


In [58]:
# {'Age': 51, 'Gender': 'Female', 'Weight_kg': 61.8, 'Height_m': 1.77, 'BMI': 19.8, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 92, 'Systolic_BP': 136, 'Diastolic_BP': 74, 'Oxygen_Saturation': 98.4, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 51, 'Gender': 'Female', 'Weight_kg': 61.8, 'Height_m': 1.77, 'BMI': 19.8, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 92, 'Systolic_BP': 136, 'Diastolic_BP': 74, 'Oxygen_Saturation': 98.4, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Walking on treadmill', 'Activity_Duration': '30-45 minutes'


In [59]:
# {'Age': 51, 'Gender': 'Female', 'Weight_kg': 61.8, 'Height_m': 1.77, 'BMI': 19.8, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 92, 'Systolic_BP': 136, 'Diastolic_BP': 74, 'Oxygen_Saturation': 98.4, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 51, 'Gender': 'Female', 'Weight_kg': 61.8, 'Height_m': 1.77, 'BMI': 19.8, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 92, 'Systolic_BP': 136, 'Diastolic_BP': 74, 'Oxygen_Saturation': 98.4, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stationary cycling', 'Activity_Duration': '30-45 minutes'


#### Female 2

In [60]:
# {'Age': 63, 'Gender': 'Female', 'Weight_kg': 85.2, 'Height_m': 1.87, 'BMI': 24.3, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 124, 'Diastolic_BP': 76, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Walking in the park', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 63, 'Gender': 'Female', 'Weight_kg': 85.2, 'Height_m': 1.87, 'BMI': 24.3, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 124, 'Diastolic_BP': 76, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Resistance band exercises', 'Activity_Duration': '30-45 minutes'


In [61]:
# {'Age': 63, 'Gender': 'Female', 'Weight_kg': 85.2, 'Height_m': 1.87, 'BMI': 24.3, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 124, 'Diastolic_BP': 76, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Walking in the park', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 63, 'Gender': 'Female', 'Weight_kg': 85.2, 'Height_m': 1.87, 'BMI': 24.3, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 124, 'Diastolic_BP': 76, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Breathing exercises', 'Activity_Duration': '30-45 minutes'


In [62]:
# {'Age': 63, 'Gender': 'Female', 'Weight_kg': 85.2, 'Height_m': 1.87, 'BMI': 24.3, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 124, 'Diastolic_BP': 76, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Walking in the park', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 63, 'Gender': 'Female', 'Weight_kg': 85.2, 'Height_m': 1.87, 'BMI': 24.3, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 124, 'Diastolic_BP': 76, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stationary cycling', 'Activity_Duration': '30-45 minutes'


### Normal. Outdoor 🏞

#### Male 1

In [63]:
input_text = "{'Age': 62, 'Gender': 'Male', 'Weight_kg': 66.6, 'Height_m': 1.89, 'BMI': 18.7, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 123, 'Diastolic_BP': 75, 'Oxygen_Saturation': 96.6, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '30-45 minutes'


In [64]:
input_text = "{'Age': 62, 'Gender': 'Male', 'Weight_kg': 66.6, 'Height_m': 1.89, 'BMI': 18.7, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 123, 'Diastolic_BP': 75, 'Oxygen_Saturation': 96.6, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '30-45 minutes'


In [65]:
input_text = "{'Age': 62, 'Gender': 'Male', 'Weight_kg': 66.6, 'Height_m': 1.89, 'BMI': 18.7, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 123, 'Diastolic_BP': 75, 'Oxygen_Saturation': 96.6, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Tai Chi in the open air', 'Activity_Duration': '30-45 minutes'


In [ ]:
#Invalid Input
input_text = "age:25, gender:male, weight:66kg, height:1.98"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

:25


In [ ]:
#Invalid Input
input_text = "age:25, gender:male, weight:66kg, height:1.98"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

#### Male 2

In [66]:
input_text = "{'Age': 49, 'Gender': 'Male', 'Weight_kg': 63.2, 'Height_m': 1.73, 'BMI': 21.2, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 70, 'Systolic_BP': 128, 'Diastolic_BP': 88, 'Oxygen_Saturation': 98.1, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Easy hiking on flat trails', 'Activity_Duration': '30-45 minutes'


In [67]:
input_text = "{'Age': 49, 'Gender': 'Male', 'Weight_kg': 63.2, 'Height_m': 1.73, 'BMI': 21.2, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 70, 'Systolic_BP': 128, 'Diastolic_BP': 88, 'Oxygen_Saturation': 98.1, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '30-45 minutes'


In [68]:
input_text = "{'Age': 49, 'Gender': 'Male', 'Weight_kg': 63.2, 'Height_m': 1.73, 'BMI': 21.2, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 70, 'Systolic_BP': 128, 'Diastolic_BP': 88, 'Oxygen_Saturation': 98.1, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Easy hiking on flat trails', 'Activity_Duration': '30-45 minutes'


#### Female 1

In [69]:
# {'Age': 51, 'Gender': 'Female', 'Weight_kg': 61.8, 'Height_m': 1.77, 'BMI': 19.8, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 92, 'Systolic_BP': 136, 'Diastolic_BP': 74, 'Oxygen_Saturation': 98.4, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 51, 'Gender': 'Female', 'Weight_kg': 61.8, 'Height_m': 1.77, 'BMI': 19.8, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 92, 'Systolic_BP': 136, 'Diastolic_BP': 74, 'Oxygen_Saturation': 98.4, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Light cycling', 'Activity_Duration': '30-45 minutes'


In [70]:
# {'Age': 51, 'Gender': 'Female', 'Weight_kg': 61.8, 'Height_m': 1.77, 'BMI': 19.8, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 92, 'Systolic_BP': 136, 'Diastolic_BP': 74, 'Oxygen_Saturation': 98.4, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 51, 'Gender': 'Female', 'Weight_kg': 61.8, 'Height_m': 1.77, 'BMI': 19.8, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 92, 'Systolic_BP': 136, 'Diastolic_BP': 74, 'Oxygen_Saturation': 98.4, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '30-45 minutes'


In [71]:
# {'Age': 51, 'Gender': 'Female', 'Weight_kg': 61.8, 'Height_m': 1.77, 'BMI': 19.8, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 92, 'Systolic_BP': 136, 'Diastolic_BP': 74, 'Oxygen_Saturation': 98.4, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 51, 'Gender': 'Female', 'Weight_kg': 61.8, 'Height_m': 1.77, 'BMI': 19.8, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 92, 'Systolic_BP': 136, 'Diastolic_BP': 74, 'Oxygen_Saturation': 98.4, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '30-45 minutes'


#### Female 2

In [72]:
input_text = "{'Age': 63, 'Gender': 'Female', 'Weight_kg': 85.2, 'Height_m': 1.87, 'BMI': 24.3, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 124, 'Diastolic_BP': 76, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '30-45 minutes'


In [73]:
input_text = "{'Age': 63, 'Gender': 'Female', 'Weight_kg': 85.2, 'Height_m': 1.87, 'BMI': 24.3, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 124, 'Diastolic_BP': 76, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Walking in the park', 'Activity_Duration': '30-45 minutes'


In [74]:
input_text = "{'Age': 63, 'Gender': 'Female', 'Weight_kg': 85.2, 'Height_m': 1.87, 'BMI': 24.3, 'BMI_Category': 'Normal', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 91, 'Systolic_BP': 124, 'Diastolic_BP': 76, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Walking in the park', 'Activity_Duration': '30-45 minutes'


### Overweight, Inddor 🏠

#### Male 1

In [75]:
# {'Age': 66, 'Gender': 'Male', 'Weight_kg': 69.1, 'Height_m': 1.65, 'BMI': 25.3, 'BMI_Category': 'Overweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 71, 'Systolic_BP': 137, 'Diastolic_BP': 82, 'Oxygen_Saturation': 96.5, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Stationary cycling', 'Activity_Duration': '20-30 minutes'}"
input_text = "{'Age': 66, 'Gender': 'Male', 'Weight_kg': 69.1, 'Height_m': 1.65, 'BMI': 25.3, 'BMI_Category': 'Overweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 71, 'Systolic_BP': 137, 'Diastolic_BP': 82, 'Oxygen_Saturation': 96.5, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Walking on treadmill', 'Activity_Duration': '20-30 minutes'


In [77]:
# {'Age': 66, 'Gender': 'Male', 'Weight_kg': 69.1, 'Height_m': 1.65, 'BMI': 25.3, 'BMI_Category': 'Overweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 71, 'Systolic_BP': 137, 'Diastolic_BP': 82, 'Oxygen_Saturation': 96.5, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Stationary cycling', 'Activity_Duration': '20-30 minutes'}"
input_text = "{'Age': 66, 'Gender': 'Male', 'Weight_kg': 69.1, 'Height_m': 1.65, 'BMI': 25.3, 'BMI_Category': 'Overweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 71, 'Systolic_BP': 137, 'Diastolic_BP': 82, 'Oxygen_Saturation': 96.5, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Resistance band exercises', 'Activity_Duration': '20-30 minutes'


### Overweight, Outdoor 🏞

In [78]:
# {'Age': 45, 'Gender': 'Female', 'Weight_kg': 74.9, 'Height_m': 1.6, 'BMI': 29.4, 'BMI_Category': 'Overweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 79, 'Systolic_BP': 114, 'Diastolic_BP': 81, 'Oxygen_Saturation': 99.4, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '20-30 minutes'}"
input_text = "{'Age': 45, 'Gender': 'Female', 'Weight_kg': 74.9, 'Height_m': 1.6, 'BMI': 29.4, 'BMI_Category': 'Overweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 79, 'Systolic_BP': 114, 'Diastolic_BP': 81, 'Oxygen_Saturation': 99.4, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '20-30 minutes'


In [79]:
# {'Age': 45, 'Gender': 'Female', 'Weight_kg': 74.9, 'Height_m': 1.6, 'BMI': 29.4, 'BMI_Category': 'Overweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 79, 'Systolic_BP': 114, 'Diastolic_BP': 81, 'Oxygen_Saturation': 99.4, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '20-30 minutes'}"
input_text = "{'Age': 45, 'Gender': 'Female', 'Weight_kg': 74.9, 'Height_m': 1.6, 'BMI': 29.4, 'BMI_Category': 'Overweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 79, 'Systolic_BP': 114, 'Diastolic_BP': 81, 'Oxygen_Saturation': 99.4, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '20-30 minutes'


In [80]:
# {'Age': 45, 'Gender': 'Female', 'Weight_kg': 74.9, 'Height_m': 1.6, 'BMI': 29.4, 'BMI_Category': 'Overweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 79, 'Systolic_BP': 114, 'Diastolic_BP': 81, 'Oxygen_Saturation': 99.4, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '20-30 minutes'}"
input_text = "{'Age': 45, 'Gender': 'Female', 'Weight_kg': 74.9, 'Height_m': 1.6, 'BMI': 29.4, 'BMI_Category': 'Overweight', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 79, 'Systolic_BP': 114, 'Diastolic_BP': 81, 'Oxygen_Saturation': 99.4, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Walking in the park', 'Activity_Duration': '20-30 minutes'


### Obese , Indoor 🏠

In [81]:
# {'Age': 51, 'Gender': 'Male', 'Weight_kg': 101.4, 'Height_m': 1.82, 'BMI': 30.6, 'BMI_Category': 'Obese', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 69, 'Systolic_BP': 112, 'Diastolic_BP': 71, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 51, 'Gender': 'Male', 'Weight_kg': 101.4, 'Height_m': 1.82, 'BMI': 30.6, 'BMI_Category': 'Obese', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 69, 'Systolic_BP': 112, 'Diastolic_BP': 71, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stationary cycling', 'Activity_Duration': '15-20 minutes'


In [82]:
# {'Age': 51, 'Gender': 'Male', 'Weight_kg': 101.4, 'Height_m': 1.82, 'BMI': 30.6, 'BMI_Category': 'Obese', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 69, 'Systolic_BP': 112, 'Diastolic_BP': 71, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 51, 'Gender': 'Male', 'Weight_kg': 101.4, 'Height_m': 1.82, 'BMI': 30.6, 'BMI_Category': 'Obese', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 69, 'Systolic_BP': 112, 'Diastolic_BP': 71, 'Oxygen_Saturation': 97.1, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '15-20 minutes'


### Obese, Outdoor 🏞

In [83]:
# {'Age': 56, 'Gender': 'Female', 'Weight_kg': 76.3, 'Height_m': 1.56, 'BMI': 31.5, 'BMI_Category': 'Obese', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 60, 'Systolic_BP': 139, 'Diastolic_BP': 74, 'Oxygen_Saturation': 99.9, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Stationary cycling', 'Activity_Duration': '15-20 minutes'}"
input_text = " {'Age': 56, 'Gender': 'Female', 'Weight_kg': 76.3, 'Height_m': 1.56, 'BMI': 31.5, 'BMI_Category': 'Obese', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 60, 'Systolic_BP': 139, 'Diastolic_BP': 74, 'Oxygen_Saturation': 99.9, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '15-20 minutes'


In [84]:
# {'Age': 56, 'Gender': 'Female', 'Weight_kg': 76.3, 'Height_m': 1.56, 'BMI': 31.5, 'BMI_Category': 'Obese', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 60, 'Systolic_BP': 139, 'Diastolic_BP': 74, 'Oxygen_Saturation': 99.9, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Stationary cycling', 'Activity_Duration': '15-20 minutes'}"
input_text = " {'Age': 56, 'Gender': 'Female', 'Weight_kg': 76.3, 'Height_m': 1.56, 'BMI': 31.5, 'BMI_Category': 'Obese', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 60, 'Systolic_BP': 139, 'Diastolic_BP': 74, 'Oxygen_Saturation': 99.9, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Walking in the park', 'Activity_Duration': '15-20 minutes'


## Stable CAD 💛

### Underweight

In [85]:
# {'Age': 66, 'Gender': 'Female', 'Weight_kg': 61.1, 'Height_m': 1.88, 'BMI': 17.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 96, 'Systolic_BP': 115, 'Diastolic_BP': 75, 'Oxygen_Saturation': 98.2, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Tai Chi in the open air', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 66, 'Gender': 'Female', 'Weight_kg': 61.1, 'Height_m': 1.88, 'BMI': 17.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 96, 'Systolic_BP': 115, 'Diastolic_BP': 75, 'Oxygen_Saturation': 98.2, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Walking in the park', 'Activity_Duration': '15-20 minutes'


In [86]:
input_text = "{'Age': 66, 'Gender': 'Female', 'Weight_kg': 61.1, 'Height_m': 1.88, 'BMI': 17.4, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 96, 'Systolic_BP': 115, 'Diastolic_BP': 75, 'Oxygen_Saturation': 98.2, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Resistance band exercises', 'Activity_Duration': '15-20 minutes'


#### Normal

In [87]:
# {'Age': 65, 'Gender': 'Male', 'Weight_kg': 63.3, 'Height_m': 1.76, 'BMI': 20.5, 'BMI_Category': 'Normal', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 87, 'Systolic_BP': 132, 'Diastolic_BP': 85, 'Oxygen_Saturation': 96.4, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 65, 'Gender': 'Male', 'Weight_kg': 63.3, 'Height_m': 1.76, 'BMI': 20.5, 'BMI_Category': 'Normal', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 87, 'Systolic_BP': 132, 'Diastolic_BP': 85, 'Oxygen_Saturation': 96.4, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Breathing exercises', 'Activity_Duration': '30-45 minutes'


In [88]:
# {'Age': 65, 'Gender': 'Male', 'Weight_kg': 63.3, 'Height_m': 1.76, 'BMI': 20.5, 'BMI_Category': 'Normal', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 87, 'Systolic_BP': 132, 'Diastolic_BP': 85, 'Oxygen_Saturation': 96.4, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 65, 'Gender': 'Male', 'Weight_kg': 63.3, 'Height_m': 1.76, 'BMI': 20.5, 'BMI_Category': 'Normal', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 87, 'Systolic_BP': 132, 'Diastolic_BP': 85, 'Oxygen_Saturation': 96.4, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Walking on treadmill', 'Activity_Duration': '30-45 minutes'


In [89]:
input_text = "{'Age': 65, 'Gender': 'Male', 'Weight_kg': 63.3, 'Height_m': 1.76, 'BMI': 20.5, 'BMI_Category': 'Normal', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 87, 'Systolic_BP': 132, 'Diastolic_BP': 85, 'Oxygen_Saturation': 96.4, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Walking in the park', 'Activity_Duration': '30-45 minutes'


In [90]:
# {'Age': 54, 'Gender': 'Female', 'Weight_kg': 72.2, 'Height_m': 1.72, 'BMI': 24.5, 'BMI_Category': 'Normal', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 75, 'Systolic_BP': 125, 'Diastolic_BP': 84, 'Oxygen_Saturation': 99.6, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Tai Chi in the open air', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 54, 'Gender': 'Female', 'Weight_kg': 72.2, 'Height_m': 1.72, 'BMI': 24.5, 'BMI_Category': 'Normal', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 75, 'Systolic_BP': 125, 'Diastolic_BP': 84, 'Oxygen_Saturation': 99.6, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Tai Chi in the open air', 'Activity_Duration': '30-45 minutes'


In [91]:
# {'Age': 54, 'Gender': 'Female', 'Weight_kg': 72.2, 'Height_m': 1.72, 'BMI': 24.5, 'BMI_Category': 'Normal', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 75, 'Systolic_BP': 125, 'Diastolic_BP': 84, 'Oxygen_Saturation': 99.6, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Tai Chi in the open air', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 54, 'Gender': 'Female', 'Weight_kg': 72.2, 'Height_m': 1.72, 'BMI': 24.5, 'BMI_Category': 'Normal', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 75, 'Systolic_BP': 125, 'Diastolic_BP': 84, 'Oxygen_Saturation': 99.6, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Easy hiking on flat trails', 'Activity_Duration': '30-45 minutes'


#### Overweight

In [92]:
# {'Age': 33, 'Gender': 'Female', 'Weight_kg': 64.3, 'Height_m': 1.54, 'BMI': 27.1, 'BMI_Category': 'Overweight', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 72, 'Systolic_BP': 133, 'Diastolic_BP': 85, 'Oxygen_Saturation': 96.7, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Tai Chi in the open air', 'Activity_Duration': '20-30 minutes'}"
input_text = "{'Age': 33, 'Gender': 'Female', 'Weight_kg': 64.3, 'Height_m': 1.54, 'BMI': 27.1, 'BMI_Category': 'Overweight', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 72, 'Systolic_BP': 133, 'Diastolic_BP': 85, 'Oxygen_Saturation': 96.7, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '20-30 minutes'


In [93]:
# {'Age': 63, 'Gender': 'Male', 'Weight_kg': 89.6, 'Height_m': 1.78, 'BMI': 28.3, 'BMI_Category': 'Overweight', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 75, 'Systolic_BP': 124, 'Diastolic_BP': 90, 'Oxygen_Saturation': 97.8, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Stationary cycling', 'Activity_Duration': '20-30 minutes'}"
input_text = "{'Age': 63, 'Gender': 'Male', 'Weight_kg': 89.6, 'Height_m': 1.78, 'BMI': 28.3, 'BMI_Category': 'Overweight', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 75, 'Systolic_BP': 124, 'Diastolic_BP': 90, 'Oxygen_Saturation': 97.8, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '20-30 minutes'


### Obese

In [94]:
# {'Age': 67, 'Gender': 'Male', 'Weight_kg': 100.2, 'Height_m': 1.72, 'BMI': 33.8, 'BMI_Category': 'Obese', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 84, 'Systolic_BP': 135, 'Diastolic_BP': 75, 'Oxygen_Saturation': 97.9, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Tai Chi in the open air', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 67, 'Gender': 'Male', 'Weight_kg': 100.2, 'Height_m': 1.72, 'BMI': 33.8, 'BMI_Category': 'Obese', 'Heart_Condition': 'Controlled Arrhythmia', 'Heart_Rate': 84, 'Systolic_BP': 135, 'Diastolic_BP': 75, 'Oxygen_Saturation': 97.9, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '15-20 minutes'


In [95]:
# {'Age': 32, 'Gender': 'Female', 'Weight_kg': 90.5, 'Height_m': 1.64, 'BMI': 33.5, 'BMI_Category': 'Obese', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 72, 'Systolic_BP': 137, 'Diastolic_BP': 72, 'Oxygen_Saturation': 99.6, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '15-20 minutes'}"
input_text = " {'Age': 32, 'Gender': 'Female', 'Weight_kg': 90.5, 'Height_m': 1.64, 'BMI': 33.5, 'BMI_Category': 'Obese', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 72, 'Systolic_BP': 137, 'Diastolic_BP': 72, 'Oxygen_Saturation': 99.6, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Easy hiking on flat trails', 'Activity_Duration': '15-20 minutes'


In [96]:
input_text = " {'Age': 32, 'Gender': 'Female', 'Weight_kg': 90.5, 'Height_m': 1.64, 'BMI': 33.5, 'BMI_Category': 'Obese', 'Heart_Condition': 'Stable CAD', 'Heart_Rate': 72, 'Systolic_BP': 137, 'Diastolic_BP': 72, 'Oxygen_Saturation': 99.6, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Breathing exercises', 'Activity_Duration': '15-20 minutes'


## Mild Pulmonary Hypertension 🟠

#### Underweight

In [98]:
# {'Age': 45, 'Gender': 'Female', 'Weight_kg': 39.8, 'Height_m': 1.53, 'BMI': 17.1, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Mild Pulmonary Hypertension', 'Heart_Rate': 82, 'Systolic_BP': 111, 'Diastolic_BP': 88, 'Oxygen_Saturation': 99.9, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Stationary cycling', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 45, 'Gender': 'Female', 'Weight_kg': 39.8, 'Height_m': 1.53, 'BMI': 17.1, 'BMI_Category': 'Underweight', 'Heart_Condition': 'Mild Pulmonary Hypertension', 'Heart_Rate': 82, 'Systolic_BP': 111, 'Diastolic_BP': 88, 'Oxygen_Saturation': 99.9, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Stationary cycling', 'Activity_Duration': '15-20 minutes'


#### Normal

In [99]:
# {'Age': 39, 'Gender': 'Female', 'Weight_kg': 60.1, 'Height_m': 1.78, 'BMI': 19.0, 'BMI_Category': 'Normal', 'Heart_Condition': 'Mild Pulmonary Hypertension', 'Heart_Rate': 77, 'Systolic_BP': 131, 'Diastolic_BP': 90, 'Oxygen_Saturation': 95.5, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Stretching exercises in a park', 'Activity_Duration': '30-45 minutes'}"
input_text = "{'Age': 39, 'Gender': 'Female', 'Weight_kg': 60.1, 'Height_m': 1.78, 'BMI': 19.0, 'BMI_Category': 'Normal', 'Heart_Condition': 'Mild Pulmonary Hypertension', 'Heart_Rate': 77, 'Systolic_BP': 131, 'Diastolic_BP': 90, 'Oxygen_Saturation': 95.5, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Walking in the park', 'Activity_Duration': '30-45 minutes'


#### Overweight

In [100]:
#  {'Age': 31, 'Gender': 'Male', 'Weight_kg': 91.8, 'Height_m': 1.79, 'BMI': 28.5, 'BMI_Category': 'Overweight', 'Heart_Condition': 'Mild Pulmonary Hypertension', 'Heart_Rate': 75, 'Systolic_BP': 117, 'Diastolic_BP': 73, 'Oxygen_Saturation': 98.3, 'Activity_Preference': 'Outdoor'},"{'Recommended_Activity': 'Tai Chi in the open air', 'Activity_Duration': '20-30 minutes'}"
input_text = "{'Age': 31, 'Gender': 'Male', 'Weight_kg': 91.8, 'Height_m': 1.79, 'BMI': 28.5, 'BMI_Category': 'Overweight', 'Heart_Condition': 'Mild Pulmonary Hypertension', 'Heart_Rate': 75, 'Systolic_BP': 117, 'Diastolic_BP': 73, 'Oxygen_Saturation': 98.3, 'Activity_Preference': 'Outdoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Tai Chi in the open air', 'Activity_Duration': '20-30 minutes'


#### Obese

In [97]:
# {'Age': 39, 'Gender': 'Female', 'Weight_kg': 100.5, 'Height_m': 1.78, 'BMI': 31.8, 'BMI_Category': 'Obese', 'Heart_Condition': 'Mild Pulmonary Hypertension', 'Heart_Rate': 62, 'Systolic_BP': 136, 'Diastolic_BP': 88, 'Oxygen_Saturation': 98.0, 'Activity_Preference': 'Indoor'},"{'Recommended_Activity': 'Yoga and stretching', 'Activity_Duration': '15-20 minutes'}"
input_text = "{'Age': 39, 'Gender': 'Female', 'Weight_kg': 100.5, 'Height_m': 1.78, 'BMI': 31.8, 'BMI_Category': 'Obese', 'Heart_Condition': 'Mild Pulmonary Hypertension', 'Heart_Rate': 62, 'Systolic_BP': 136, 'Diastolic_BP': 88, 'Oxygen_Saturation': 98.0, 'Activity_Preference': 'Indoor'}"

# Tokenize input dan pindahkan ke GPU
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

# Generate output dari model yang sudah di-fine-tune
output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

'Recommended_Activity': 'Breathing exercises', 'Activity_Duration': '15-20 minutes'


# Model Saving 💾

In [101]:
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/spiece.model',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [103]:
from google.colab import files
import shutil

shutil.make_archive('fine_tuned_model', 'zip', './fine_tuned_model')
files.download('fine_tuned_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>